In [95]:
# https://stackoverflow.com/questions/33289820/noun-phrases-with-spacy

In [25]:
from reader import *

ModuleNotFoundError: No module named 'reader'

In [2]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 17.4MB/s eta 0:00:011
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.0-cp37-none-any.whl size=98072933 sha256=8eef9f4f668dae59a89ef821d02ae505e2654d1d1bb3ea6a749475c696b050bf
  Stored in directory: /private/var/folders/qj/vhjml8kj1tl0shjyxg0rfq5c0000gn/T/pip-ephem-wheel-cache-_13z7upu/wheels/5f/3e/c9/36dd6e13b449fd84cd1f94b72dfbc559daf09f53dbf4e697a3
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [2]:
import spacy
nlp = spacy.load("en_core_web_md")

In [3]:
!export SPACY_WARNING_IGNORE=W008

In [4]:
input_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/a10.input'
key_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/a10.key'

In [5]:
reader = ReadInput(input_path)
list_of_sentences = reader.getListOfSentences()
# print(list_of_sentences)
full_text = reader.getText()
# print(full_text)

In [6]:
sentence_dict = getSentenceDict(list_of_sentences)
cluster_head_dict = getClusterHeads(sentence_dict)

In [7]:
def getRemText(sent_id, sentence_dict):
    rem_text = {}
    for sid in sentence_dict.keys():
        if(sid >= sent_id): 
            rem_text[sid] = sentence_dict[sid]
    sorted_rem_text = dict(sorted(rem_text.items(), key = lambda x:x[0]))
    return sorted_rem_text

In [8]:
def getCorefDict_max_of_each_sentence(sentence_dict, cluster_head_dict, threshold):
    '''
    This function gives the most matching word in each sentence for a given cluster head
    It returns a dictionary:
        - Key: Cluster_head_id
        - Value: list of (current_cluster_head_name, sid, np, similarity_score)
    '''
    
    ## Initialise a coref_dict 
    coref_dict = {}
    
    for cluster_id, cluster_value in cluster_head_dict.items():
        current_cluster_sent_id = cluster_value[0]        
        current_cluster_head = cluster_value[1]
        
        remaining_text = getRemText(current_cluster_sent_id, sentence_dict)

        for sid, sentence in remaining_text.items():
            clean = re.compile('<COREF .*?>.*?</COREF>') ## removing all coref tags from the current sentence
            clean_sentence = re.sub(clean, '', sentence)

            doc = nlp(clean_sentence)
            word_embedding_score_list_per_sentence = []
            
            for np in doc.noun_chunks:
                
                ## If the NP is a pronoun, ignore it. 
                ## We are handling pronouns by Hobb's algorithm
                if(len(np) == 1 and np[0].pos_ == 'PRON'):
                    continue
                
                similarity_score = nlp(current_cluster_head).similarity(np)
                
                               
                word_embedding_score_list_per_sentence.append(list([current_cluster_head, sid, np, similarity_score]))
            
            if(len(word_embedding_score_list_per_sentence) == 0):
                continue
            scores_list = list(map(lambda x: x[3], word_embedding_score_list_per_sentence))
            max_score = max(scores_list)
            max_index = scores_list.index(max_score)

            max_word_embedding_score_list_per_sentence = word_embedding_score_list_per_sentence[max_index]


            if(max_score > threshold):
                if(cluster_id not in coref_dict.keys()):
                    coref_dict[cluster_id] = max_word_embedding_score_list_per_sentence
                else:
                    coref_dict[cluster_id].append(max_word_embedding_score_list_per_sentence)

    return coref_dict

In [9]:
def getCorefDict_all_sorted(sentence_dict, cluster_head_dict, threshold):
    '''
    This function gives the all the matching words in the whole doc, in desc order, for a given cluster head
    It returns a dictionary:
        - Key: Cluster_head_id
        - Value: list of (current_cluster_head_name, sid, np, similarity_score)
    '''
    
    ## Initialise a coref_dict 
    coref_dict = {}
    
    for cluster_id, cluster_value in cluster_head_dict.items():
        current_cluster_sent_id = cluster_value[0]        
        current_cluster_head = cluster_value[1]
        
        remaining_text = getRemText(current_cluster_sent_id, sentence_dict)
        
        word_embedding_score_list_all_sentences = []

        for sid, sentence in remaining_text.items():
            clean = re.compile('<COREF .*?>.*?</COREF>') ## removing all coref tags from the current sentence
            clean_sentence = re.sub(clean, '', sentence)

            doc = nlp(clean_sentence)

            for np in doc.noun_chunks:
                
                ## If the NP is a pronoun, ignore it. 
                ## We are handling pronouns by Hobb's algorithm
                if(len(np) == 1 and np[0].pos_ == 'PRON'):
                    continue
                
                similarity_score = nlp(current_cluster_head).similarity(np)
                
                if(similarity_score > threshold):
                    word_embedding_score_list_all_sentences.append(list([current_cluster_head, sid, np, similarity_score]))
                                
                
        if(len(word_embedding_score_list_all_sentences) == 0):
            continue
        else:
            scores_list = list(map(lambda x: x[3], word_embedding_score_list_all_sentences))
            sorted_indices = [i[0] for i in sorted(enumerate(scores_list), key=lambda x:x[1], reverse=True)]
            sorted_word_embedding_score_list_all_sentences = [word_embedding_score_list_all_sentences[i] for i in sorted_indices]


        if(cluster_id not in coref_dict.keys()):
            coref_dict[cluster_id] = sorted_word_embedding_score_list_all_sentences
        else:
            coref_dict[cluster_id].append(sorted_word_embedding_score_list_all_sentences)         

    return coref_dict

In [23]:
def get_TopN_Matches(coref_dict_all_sorted,n):
    '''
    To get Top N word embeddings in the document per Cluster Head
    Modifies and returns coref_dict_all_sorted
    '''
    
    for cluster_id, word_embedding_list in coref_dict_all_sorted.items():
        coref_dict_all_sorted[cluster_id] = coref_dict_all_sorted[cluster_id][:n]
    
    return coref_dict_all_sorted

In [10]:
coref_dict_all_sorted = getCorefDict_all_sorted(sentence_dict, cluster_head_dict,0.5)

/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4

In [16]:
coref_dict_max_sentence = getCorefDict_max_of_each_sentence(sentence_dict, cluster_head_dict,0.5)

/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4

In [12]:
reader_o = ReadInput(key_path)
ans = reader_o.getListOfSentences()

In [13]:
ans

['<COREF ID="X0">GROUNDED-JETS</COREF>\n',
 '{5} {GROUNDED-JETS} {JETS}\n',
 '{21} {F-14s} {F-14s}\n',
 '{25} {the F-14s} {F-14s}\n',
 '\n',
 '<COREF ID="X1">02-22</COREF>\n',
 '{38} {02-22-96} {02-22-96}\n',
 '\n',
 '<COREF ID="X2">NAVY</COREF>\n',
 '{10} {Navy} {Navy}\n',
 '{12} {Navy} {Navy}\n',
 '{13} {Navy} {Navy}\n',
 '{15} {Navy} {Navy}\n',
 '{19} {Navy} {Navy}\n',
 '{21} {Navy} {Navy}\n',
 '{27} {Navy} {Navy}\n',
 '{30} {Navy} {Navy}\n',
 '{35} {Navy} {Navy}\n',
 '{36} {Navy} {Navy}\n',
 '{36} {they} {they}\n',
 '\n',
 '<COREF ID="X3">THIRD RECENT CRASH</COREF>\n',
 "{10} {the third ``Tomcat'' crash in a month} {crash}\n",
 '{10} {the second in a week} {second}\n',
 '{19} {the crash} {crash}\n',
 '{19} {the third Tomcat crash} {crash}\n',
 '{26} {the latest crash} {crash}\n',
 '\n',
 '<COREF ID="X4">entire fleet of F-14 airplanes</COREF>\n',
 '{26} {the entire fleet of more than 300 aircraft} {fleet}\n',
 '\n',
 '<COREF ID="X5">Thursday</COREF>\n',
 '{19} {Thursday} {Thursday}\

In [14]:
coref_dict_all_sorted

{'X0': [['GROUNDED-JETS', 5, BC-GROUNDED-JETS-BOS, 0.7913340019900723],
  ['GROUNDED-JETS', 31, twin-engine aircraft, 0.583906469123143],
  ['GROUNDED-JETS', 31, close-in air combat, 0.5454798322739851],
  ['GROUNDED-JETS', 23, aircraft, 0.5080282907209213]],
 'X2': [['NAVY', 10, The Navy, 0.8559929223901659],
  ['NAVY', 12, The Navy, 0.8559929223901659],
  ['NAVY', 13, The Navy, 0.8559929223901659],
  ['NAVY', 15, The Navy, 0.8559929223901659],
  ['NAVY', 19, the Navy, 0.8559929223901659],
  ['NAVY', 27, the Navy, 0.8559929223901659],
  ['NAVY', 30, The Navy, 0.8559929223901659],
  ['NAVY', 31, the Navy, 0.8559929223901659],
  ['NAVY', 35, the Navy, 0.8559929223901659],
  ['NAVY', 36, The Navy, 0.8559929223901659],
  ['NAVY', 21, Navy personnel, 0.8145926717548846],
  ['NAVY', 29, the Navy's first female pilots, 0.5898204596145711],
  ['NAVY', 26, Naval Operations, 0.523784133108768],
  ['NAVY', 34, Naval records, 0.5231776250941521]],
 'X3': [['THIRD RECENT CRASH', 26, the latest cra

In [17]:
coref_dict_max_sentence

{'X0': ['GROUNDED-JETS',
  5,
  BC-GROUNDED-JETS-BOS,
  0.7913340019900723,
  ['GROUNDED-JETS', 23, aircraft, 0.5080282907209213],
  ['GROUNDED-JETS', 31, twin-engine aircraft, 0.583906469123143]],
 'X2': ['NAVY',
  10,
  The Navy,
  0.8559929223901659,
  ['NAVY', 12, The Navy, 0.8559929223901659],
  ['NAVY', 13, The Navy, 0.8559929223901659],
  ['NAVY', 15, The Navy, 0.8559929223901659],
  ['NAVY', 19, the Navy, 0.8559929223901659],
  ['NAVY', 21, Navy personnel, 0.8145926717548846],
  ['NAVY', 26, Naval Operations, 0.523784133108768],
  ['NAVY', 27, the Navy, 0.8559929223901659],
  ['NAVY', 29, the Navy's first female pilots, 0.5898204596145711],
  ['NAVY', 30, The Navy, 0.8559929223901659],
  ['NAVY', 31, the Navy, 0.8559929223901659],
  ['NAVY', 34, Naval records, 0.5231776250941521],
  ['NAVY', 35, the Navy, 0.8559929223901659],
  ['NAVY', 36, The Navy, 0.8559929223901659]],
 'X3': ['THIRD RECENT CRASH',
  7,
  New York Times News Service clients,
  0.5694651929839202,
  ['THIRD R